In [1]:
!pip3 install pika

In [2]:
import pika

Тут мы объявляем метод для обработки полученного сообщения и для получения сообщений.
Не забудьте указать корректные логин, пароль и корректное название очереди

In [6]:
import json
from time import sleep

def process_message(ch, method, properties, body):
    try:
        # Парсинг JSON-сообщения
        message = json.loads(body)
        print(f"Получено сообщение: {message}")
        sleep(1)
        # Пример обработки сообщения
        if 'service' in message:
            print(f"Обработка данных для сервиса: {message['service']}")
    except json.JSONDecodeError:
        print("Ошибка: Невозможно распознать сообщение как JSON")
    finally:
        # Подтверждаем получение сообщения
        ch.basic_ack(delivery_tag=method.delivery_tag)



def receive_message():
    # Учетные данные для авторизации
    credentials = pika.PlainCredentials('user', 'password')  # Используйте вашего пользователя и пароль
    connection_params = pika.ConnectionParameters(
        host='94.131.105.157',       # Хост кластера
        port=5672,              # Порт AMQP
        credentials=credentials # Учетные данные для подключения
    )

    # Устанавливаем соединение с RabbitMQ
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Указываем, что будем получать сообщения из очереди 'test_queue'
    channel.basic_consume(
        queue='test_queue',
        on_message_callback=process_message
    )

    print("Ожидание сообщений. Нажмите Ctrl+C для завершения.")
    # Начинаем приём сообщений
    channel.start_consuming()

А здесь включаем сам консьюмер

In [7]:
receive_message()

Ожидание сообщений. Нажмите Ctrl+C для завершения.
Получено сообщение: Сообщение с номером 1
Получено сообщение: Сообщение с номером 3
Получено сообщение: Сообщение с номером 5
Получено сообщение: Сообщение с номером 7
Получено сообщение: Сообщение с номером 9
Получено сообщение: Сообщение с номером 11
Получено сообщение: Сообщение с номером 13
Получено сообщение: Сообщение с номером 15
Получено сообщение: Сообщение с номером 17
Получено сообщение: Сообщение с номером 19
Получено сообщение: Сообщение с номером 21
Получено сообщение: Сообщение с номером 23
Получено сообщение: Сообщение с номером 25
Получено сообщение: Сообщение с номером 1
Получено сообщение: Сообщение с номером 3
Получено сообщение: Сообщение с номером 83
Получено сообщение: Сообщение с номером 84
Получено сообщение: Сообщение с номером 85
Получено сообщение: Сообщение с номером 86
Получено сообщение: Сообщение с номером 87
Получено сообщение: Сообщение с номером 88
Получено сообщение: Сообщение с номером 89
Получено с

StreamLostError: Transport indicated EOF